In [1]:
# x.py
from helper import *
from nse_func import *

import nest_asyncio
nest_asyncio.apply()

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('nse', 'live')

with open(logpath+'ztest.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'ztest.log')

In [ ]:
from IPython.core.display import SVG
SVG(filename='../pic/'+'nse_scrape.svg')

In [2]:
def get_lots():
    '''Get the lots with expiry dates
    Arg: None
    Returns: lots dataframe with expiry as YYYYMM''' 

    url = 'https://www.nseindia.com/content/fo/fo_mktlots.csv'
    req = requests.get(url)
    data = StringIO(req.text)
    lots_df = pd.read_csv(data)

    lots_df = lots_df[list(lots_df)[1:5]]

    # strip whitespace from columns and make it lower case
    lots_df.columns = lots_df.columns.str.strip().str.lower() 

    # strip all string contents of whitespaces
    lots_df = lots_df.applymap(lambda x: x.strip() if type(x) is str else x)

    # remove 'Symbol' row
    lots_df = lots_df[lots_df.symbol != 'Symbol']

    # melt the expiries into rows
    lots_df = lots_df.melt(id_vars=['symbol'], var_name='expiryM', value_name='lot').dropna()

    # remove rows without lots
    lots_df = lots_df[~(lots_df.lot == '')]

    # convert expiry to period
    lots_df = lots_df.assign(expiryM=pd.to_datetime(lots_df.expiryM, format='%b-%y').dt.to_period('M'))

    # convert lots to integers
    lots_df = lots_df.assign(lot=pd.to_numeric(lots_df.lot, errors='coerce'))
    
    # convert & to %26
    lots_df = lots_df.assign(symbol=lots_df.symbol.str.replace('&', '%26'))

    return lots_df.reset_index(drop=True)

def get_xu(symbol: str) -> pd.DataFrame():
    '''Gets the symbol, expiry, undPrice'''
    # get expiries for the symbol
    url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol='
    xpd = "//*[@id='date']" # xpath for date select options
    xpu = "//*[@id='wrapper_btm']/table[1]/tr/td[2]/div/span[1]/b" # xpath for undPrice
    
    res = requests.get(url + symbol).text
    htree = html.fromstring(res) #html is from lxml 
    expiries = [opt.text for e in htree.xpath(xpd) for opt in e if 'Select' not in opt.text.strip('')]
    undPrice = [float(e.text.split(' ')[1]) for e in htree.xpath(xpu)][0]

    # convert above to a DataFrame
    df = pd.DataFrame(list(product([symbol], expiries, [str(undPrice)])), 
                      columns=['symbol', 'expiry', 'undPrice'])

    return df.apply(pd.to_numeric, errors = 'ignore')

def get_nse_chain(symbol: str, expiry: 'datetime64', undPrice: float, lot: int) -> pd.DataFrame:
    '''gets option chain for nse'''
    
    url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbol='
    
    u = url+symbol+'&date='+expiry

    chainhtml = requests.get(u, headers=headers).content
    chain = pd.read_html(chainhtml)[1][:-1]  # read the first table and drop the total
    chain.columns=chain.columns.droplevel(0) # drop the first row of the header
    chain = chain.drop('Chart', 1)           # drop the charts
    
    cols = ['pOI', 'pOI_Chng', 'pVolume', 'pIV', 'pLTP', 'pNetChng', 'pBidQty', 'pBid', 'pAsk', 'pAskQty',
             'strike', 'cBidQty', 'cBid', 'cAsk', 'cAskQty', 'cNetChng', 'cLTP', 'cIV', 'cVolume', 'cOI_Chng', 'cOI']

    # rename the columns
    chain.columns = cols

    chain = chain.iloc[2:] # remove the first two rows

    # convert all to numeric
    chain = chain.apply(pd.to_numeric, errors = 'coerce')
    chain.insert(0, 'symbol', symbol)
    chain.insert(1, 'expiry', datetime.datetime.strptime(expiry, '%d%b%Y').date())
    chain.insert(2, 'undPrice', undPrice)
    chain.insert(3, 'lot', lot)
    
    return chain

def do_hist(ib, undId):
    '''Historize ohlc
    Args:
        (ib) as connection object
        (undId) as contractId for underlying symbol in int
    Returns:
        df_hist as dataframe
        pickles the dataframe by symbol name
    '''
    qc = ib.qualifyContracts(Contract(conId=int(undId)))[0]
    hist = ib.reqHistoricalData(contract=qc, endDateTime='', 
                                        durationStr='365 D', barSizeSetting='1 day',  
                                                    whatToShow='Trades', useRTH=True)
    df_hist = util.df(hist)
    df_hist = df_hist.assign(date=pd.to_datetime(df_hist.date, format='%Y-%m-%d'))
    df_hist.insert(loc=0, column='symbol', value=qc.symbol)
    df_hist = df_hist.sort_values('date', ascending = False).reset_index(drop=True)
    df_hist.to_pickle(fspath+'_'+qc.symbol+'_ohlc.pkl')
    return None

In [ ]:
# get_dte.py
def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format 'yyyymmdd'
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

In [ ]:
%%time

#... Make the chains

# get the symbols and lots
df_lots = get_lots()
symbols = sorted(list(df_lots.symbol.unique()))

# symbols = [s for s in nse_symbols if s in ['NIFTY', 'PNB']] # DATA LIMITER!!!
# symbols = nse_symbols[:5] # DATA LIMITER!!!

# get the strikes, expiry and undPrices
df_sxu = pd.concat([get_xu(s) for s in symbols]).reset_index(drop=True)
df_sxu = df_sxu.assign(expiry=pd.to_datetime(df_sxu.expiry))

# get the lots
df_sxul = df_sxu.assign(expiryM=df_sxu.expiry.dt.to_period('M')).merge(df_lots).drop('expiryM', 1)

# convert expiry to nse friendly date
df_sxul = df_sxul.assign(expiry=[f"{dt.day}{calendar.month_abbr[dt.month].upper()}{dt.year}" for dt in df_sxul.expiry])

tqr = tnrange(len(df_sxul), desc='Processing', leave=True)
chains = []
for symbol, expiry, undPrice, lot in zip(*[df_sxul[col] for col in df_sxul.columns]):
    tqr.set_description(f"Processing [{symbol}]")
    tqr.refresh() # to show immediately the update
    chains.append(catch(lambda: get_nse_chain(symbol, expiry, undPrice, lot)))
    tqr.update(1)
tqr.close()

# remove empty elements in list of dfs and concatenate
df_chains = pd.concat([x for x in chains if str(x) != 'nan'])

# remove nan from prices
df_chains = df_chains.dropna(subset=['cBid', 'cAsk', 'cLTP', 'pBid', 'pAsk', 'pLTP']).reset_index(drop=True)

# convert symbols - friendly to IBKR and pickle the chains
df_chains = df_chains.assign(symbol=df_chains.symbol.str.slice(0,9))

ntoi = {'M%26M': 'MM', 'M%26MFIN': 'MMFIN', 'L%26TFH': 'LTFH', 'NIFTY': 'NIFTY50', 'CHOLAFIN':'CIFC'}
df_chains.symbol = df_chains.symbol.replace(ntoi)

# set the types for indexes as IND
ix_symbols = ['NIFTY50', 'BANKNIFTY', 'NIFTYIT']

# build the underlying contracts
scrips = list(df_chains.symbol.unique())
und_contracts = [Index(symbol=s, exchange=exchange) if s in ix_symbols else Stock(symbol=s, exchange=exchange) for s in scrips]

# get the underlying conIds
qual_unds = ib.qualifyContracts(*und_contracts)
df_chains = df_chains.assign(undId = df_chains.symbol.map({q.symbol: q.conId for q in qual_unds}))

# convert datetime to correct format for IB
df_chains = df_chains.assign(expiry=[e.strftime('%Y%m%d') for e in df_chains.expiry])

df_chains.to_pickle(fspath+'nse_chains.pkl')

#... Make the ohlcs with stDev

# historize individual underlyings
[catch(lambda: do_hist(ib, uid)) for uid in list(df_chains.undId.unique())]

# Capture the ohlc pickles
ohlc_pkls = [f for f in listdir(fspath) if (f[-8:] == 'ohlc.pkl')]
df_ohlcs = pd.concat([pd.read_pickle(fspath+o) for o in ohlc_pkls]).reset_index(drop=True)

# put stdev in ohlc
df_ohlcs = df_ohlcs.assign(stDev=df_ohlcs.groupby('symbol').close.transform(lambda x: x.expanding(1).std(ddof=0)))
df_ohlcs.to_pickle(fspath+'ohlcs.pkl')

In [49]:
# Temporary code to read pickle files
df_chains = pd.read_pickle(fspath+'nse_chains.pkl')
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

In [65]:
%%time
#... Remove chains not meeting put and call std filter

# get dte
df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))

# generate std dataframe
df = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
df = df.assign(dte=df.groupby('symbol').cumcount()) # get the cumulative count for location as dte
df.set_index(['symbol', 'dte'])

df1 = df_chains[['symbol', 'dte']]  # data to be looked at
df2 = df1.drop_duplicates()  # make data to be looked at smaller

df_std = df2.set_index(['symbol', 'dte']).join(df.set_index(['symbol', 'dte']))

# join to get std in chains
df_chainstd = df_chains.set_index(['symbol', 'dte']).join(df_std).reset_index()

# columns for puts and calls
putcols = [c for c in list(df_chainstd) if c[0] != 'c']
callcols = [p for p in list(df_chainstd) if p[0] != 'p']

# make puts and calls dataframe with std filter
df_puts = df_chainstd[df_chainstd.strike < (df_chainstd.undPrice-(df_chainstd.stDev*putstdmult))][putcols]
df_puts = df_puts.assign(right = 'P')

df_calls = df_chainstd[df_chainstd.strike > (df_chainstd.undPrice+(df_chainstd.stDev*callstdmult))][callcols]
df_calls = df_calls.assign(right = 'C')

# rename puts columns by removing 'p'
df_puts = df_puts.rename(columns={p: p[1:] for p in putcols if p[0] == 'p'})

# rename calls calumns by removing 'c'
df_calls = df_calls.rename(columns= {c: c[1:] for c in callcols if c[0] == 'c'})

df_opt = pd.concat([df_puts, df_calls], sort=False).reset_index(drop=True)

Wall time: 96.7 ms


In [ ]:
%%time
#...get the margins

# ... get the conId of the option contracts
optipl = [Option(s, util.formatIBDatetime(e), k, r, exchange) for s, e, k, r in zip(df_opt.symbol, df_opt.expiry, df_opt.strike, df_opt.right)]
optblks = [optipl[i: i+blk] for i in range(0, len(optipl), blk)] # blocks of optipl

# qualify the contracts
cblks = [ib.qualifyContracts(*s) for s in optblks]
contracts = [z for x in cblks for z in x]

# extract contractId for the options
df_conId = util.df(contracts).iloc[:, 1:6]
df_conId = df_conId.rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

df_conId = df_conId.rename(columns={'conId':'optId'})

df_opt = pd.merge(df_opt, df_conId, how='left', on=['symbol', 'expiry', 'strike', 'right'])

# build contracts and orders
opt_con_dict = {c.conId: c for c in contracts}
opt_ord_dict = {i: Order(action='SELL', orderType='MKT', totalQuantity=lot) for i, lot in zip(df_opt.optId, df_opt.lot)}

In [85]:
%%time
# combine contracts and orders
opts = zip(df_opt.optId.map(opt_con_dict), df_opt.optId.map(opt_ord_dict))

Wall time: 77.8 ms


In [86]:
dict_margins = {c: float(ib.whatIfOrder(c, o).initMarginChange) for c, o in opts}

KeyboardInterrupt: 

In [84]:
list(opts)


[]

In [72]:
# gather and get the margins through asyncio
g = asyncio.gather(*[getMarginAsync(ib, c, o) for c, o in opts])
opt_mgn_dict = {k: v for d in asyncio.run(g) for k, v in d.items()}

df_opt = df_opt.assign(optMargin=df_opt.optId.map(opt_mgn_dict)).fillna(df_opt.undMargin)

AttributeError: 'DataFrame' object has no attribute 'undMargin'

In [74]:
g

<_GatheringFuture finished result=[{359168000: nan}, {359177689: nan}, {359177723: nan}, {359177736: nan}, {359177747: nan}, {359177753: nan}, ...]>